In [1]:
import os
import sys

from analysis import *
import matplotlib.pyplot as plt
import numpy as np

print("Setup completed successfully")

Setup completed successfully


# Test influence of number of moves to solution found

In [9]:
def test_scramble_moves_influence(max_n=30):
    n_values = range(0, max_n + 1)
    scores = []
    
    for n in n_values:
        score, _, _ = test_solver(time_limit=1000, search_depth=4, search_depth_g1=5, num_tests=100, num_moves=n)
        scores.append(score)
    
    plt.figure(figsize=(12, 8)) 
    plt.plot(n_values, scores, marker='o')
    
    for i, score in enumerate(scores):
        plt.annotate(f'{score:.2f}', (n_values[i], score), textcoords="offset points", 
                     xytext=(0,10), ha='center', fontsize=10)
    
    plt.title("Vliv počtu tahů při míchání na dosažené skóre")
    plt.xlabel("Počet tahů míchání (n)")
    plt.ylabel("Dosažené skóre (průměr ze 100 testů)")
    plt.grid(True)
    
    y_min, y_max = plt.ylim()
    plt.ylim(y_min, y_max + (y_max - y_min) * 0.1)
    
    plt.tight_layout()  
    plt.savefig("scramble_moves_influence.png", dpi=300, bbox_inches="tight")
    plt.show()


test_scramble_moves_influence(max_n=30)

Failed to build the project
stdout: 
stderr: Error: cabal: There is no <pkgname>.cabal package file or cabal.project file.
To build packages locally you need at minimum a <pkgname>.cabal file. You can
use 'cabal init' to create one.

For non-trivial projects you will also want a cabal.project file in the root
directory of your project. This file lists the packages in your project and
all other build configuration. See the Cabal user guide for full details.


Test 1/100
Shuffle: 
Stderr: Final score: 120.0

solution: [], score: 120.0, duration: 0.0721137523651123
Test 2/100
Shuffle: 
Stderr: Final score: 120.0

solution: [], score: 120.0, duration: 0.05196690559387207
Test 3/100
Shuffle: 
Stderr: Final score: 120.0

solution: [], score: 120.0, duration: 0.06215643882751465
Test 4/100
Shuffle: 
Stderr: Final score: 120.0

solution: [], score: 120.0, duration: 0.06227588653564453
Test 5/100
Shuffle: 
Stderr: Final score: 120.0

solution: [], score: 120.0, duration: 0.06366848945617676
Tes

KeyboardInterrupt: 